In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import datetime

import matplotlib.pyplot as plt

from scipy import stats
import math
import random

from sklearn import preprocessing
from sklearn.model_selection import ShuffleSplit, train_test_split, cross_val_score, StratifiedShuffleSplit
from sklearn.metrics import  mean_squared_log_error

from xgboost import XGBRegressor

In [60]:
#Global Variables for grid search
n_splits = 10
n_jobs = 2

max_depth_min = 3
max_depth_max = 11
n_estimator_min = 100
n_estimator_max = 200

test_size =0.5
random_state = 1986

In [61]:
train_File = '../input/train.csv'
test_File = '../input/test.csv'

df_train = pd.read_csv(train_File)
df_test = pd.read_csv(test_File)
df_test['SalePrice'] = 0
df_concat = pd.concat([df_train,df_test])

In [62]:
df_concat.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,...,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,...,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,...,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,...,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,...,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [63]:
def giveMeWrangledData(df, testFile=False, log=False):
    
    
    df = df.drop(['Id', 'GarageYrBlt','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF'],axis=1)
    
    df['LotFrontage'] =df.LotFrontage.fillna(df.LotFrontage.mode()[0])
    df['MasVnrArea']=df.MasVnrArea.fillna(0.0)
    df['TotalBsmtSF'] = df.TotalBsmtSF.fillna(0)
    df['BsmtFullBath'] = df.BsmtFullBath.fillna(0)
    df['BsmtHalfBath'] = df.BsmtHalfBath.fillna(0)
    df['GarageCars'] = df.GarageCars.fillna(0)
    df['GarageArea'] = df.GarageArea.fillna(0)
    
    #convert data type
    #we are being little lineant to give int64 for YearBuilt, YrSold but those guys are going to be box-coxed 
    #so let them at least enjoy the bigger size for now
    int64_variables = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', \
                     'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', \
                     'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',\
                     'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', \
                     'PoolArea', 'MiscVal', 'YrSold', 'SalePrice']
    
    #if testFile:
    #    int64_variables.remove('SalePrice')

        
        
        
    
    for c in int64_variables:
        if log:
            print("Changing the data type for :", c)
        #df[c] = df[c].astype(np.int64)
        df[c] = df[c].astype(np.float64)
        
    int_to_categorical_variables = ['MSSubClass', 'OverallQual', 'OverallCond', 'FireplaceQu', 'MoSold']
    for c in int_to_categorical_variables:
        df[c] = df[c].astype(str)
        
    df = df.fillna('NotAvailable')
    return df
df = giveMeWrangledData(df_concat)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450.0,Pave,NotAvailable,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003.0,2003.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,856.0,GasA,Ex,Y,SBrkr,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,Gd,8.0,Typ,0.0,nan,Attchd,RFn,2.0,548.0,TA,TA,Y,0.0,61.0,0.0,0.0,0.0,0.0,NotAvailable,NotAvailable,NotAvailable,0.0,2,2008.0,WD,Normal,208500.0
1,20,RL,80.0,9600.0,Pave,NotAvailable,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976.0,1976.0,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,1262.0,GasA,Ex,Y,SBrkr,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,TA,6.0,Typ,1.0,TA,Attchd,RFn,2.0,460.0,TA,TA,Y,298.0,0.0,0.0,0.0,0.0,0.0,NotAvailable,NotAvailable,NotAvailable,0.0,5,2007.0,WD,Normal,181500.0
2,60,RL,68.0,11250.0,Pave,NotAvailable,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001.0,2002.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,920.0,GasA,Ex,Y,SBrkr,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,Gd,6.0,Typ,1.0,TA,Attchd,RFn,2.0,608.0,TA,TA,Y,0.0,42.0,0.0,0.0,0.0,0.0,NotAvailable,NotAvailable,NotAvailable,0.0,9,2008.0,WD,Normal,223500.0
3,70,RL,60.0,9550.0,Pave,NotAvailable,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915.0,1970.0,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,756.0,GasA,Gd,Y,SBrkr,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,Gd,7.0,Typ,1.0,Gd,Detchd,Unf,3.0,642.0,TA,TA,Y,0.0,35.0,272.0,0.0,0.0,0.0,NotAvailable,NotAvailable,NotAvailable,0.0,2,2006.0,WD,Abnorml,140000.0
4,60,RL,84.0,14260.0,Pave,NotAvailable,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000.0,2000.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,1145.0,GasA,Ex,Y,SBrkr,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,Gd,9.0,Typ,1.0,TA,Attchd,RFn,3.0,836.0,TA,TA,Y,192.0,84.0,0.0,0.0,0.0,0.0,NotAvailable,NotAvailable,NotAvailable,0.0,12,2008.0,WD,Normal,250000.0


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 76 columns):
MSSubClass       2919 non-null object
MSZoning         2919 non-null object
LotFrontage      2919 non-null float64
LotArea          2919 non-null float64
Street           2919 non-null object
Alley            2919 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
Utilities        2919 non-null object
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null object
OverallCond      2919 non-null object
YearBuilt        2919 non-null float64
YearRemodAdd     2919 non-null float64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd    

In [65]:
def preProcessData(df, log=False):
    
    

    #get dummies
    if log:
        print("Shape of the data set before pre processing : ", df.shape )
        print("Categorical columns : ", list(df.select_dtypes(exclude=np.number)))
    df = pd.get_dummies(df, dtype=np.float64)
    
    
    
    if log:
        print("\n\nShape of the data set after pre processing : ", df.shape )
        print("Columns in the data set are : ",list(df))

    return df

df_prep = preProcessData(df)
df_prep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Columns: 351 entries, LotFrontage to SaleCondition_Partial
dtypes: float64(351)
memory usage: 7.8 MB


https://stats.stackexchange.com/questions/130262/why-not-log-transform-all-variables-that-are-not-of-main-interest

In [70]:
from IPython import display
def transformTarget(df_temp, revert=False ):
    '''df = df_temp.copy()
    #df['new_variable'] = df.BedroomAbvGr * .1 + df.FullBath *.25 + df.HalfBath * .5 + df.BsmtFullBath *.75 + df.BsmtHalfBath * 1
    #df['new_variable'] = df.BedroomAbvGr * .1 + df.FullBath *.25 + df.HalfBath * .5 + df.BsmtFullBath *.75 + df.BsmtHalfBath * .1
    df['new_variable'] = df.BedroomAbvGr.apply(lambda x: x if x > 0 else 1)
    #print("new_variable calculated")
    #display.display(df[df.new_variable.isna()])
    
    if not revert:
        df['sales_per_new'] = df['SalePrice']/ df.new_variable
        #display.display(df[df.sales_per_new.isna()])
        y = np.array(df['sales_per_new'].apply( lambda x: math.log(x)))
        return y
    
    df['sales_per_new'] = df['SalePrice']* df.new_variable
    #print("sales_per_new calculated")
    #display.display(df[df.sales_per_new.isna()])'''
    
    return np.log(df_temp['SalePrice'])
    

In [ ]:
def newBoxCoxTranformation(df_temp,target,testFile=False, log=False):
    df = df_temp.copy()
    #assuming that only numerical features are presented
    if log:
        print("Shape of the dataset initial : ", df.shape)
    
    if not testFile:
        df =df[df.SalePrice >0]
        if log:
            print("Shape of the dataset before transformation : ", df.shape)
        
        y = transformTarget(df)
        X= df.drop([target],axis = 1)
        #X=df.drop(target, axis=1)
        x_columns = list(X)
        X = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X)
        X_testx = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X,y)
        X_testxx = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X,df.SalePrice)
        print(np.unique(X == X_testx))
        print(np.unique(X == X_testxx))
        print(np.unique(X_testx == X_testxx))
        X = pd.DataFrame(X, columns=x_columns)
        
        for c in list(X):
            if len(X[c].unique()) in  [1,2]:
                if log:
                    print("Skipping Transformation for ", c, "because unique values are :",X[c].unique())
            else:
                if log:
                    print("Boxcoxing : ", c)
                X[c] = stats.boxcox(X[c])[0]
        
        X = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X)
        #X = X.values
        if log:
            print("Shape of the dataset after transformation : ", X.shape, y.shape)
        return X,y
    else:
        df = df[df.SalePrice == 0.0]
        if log:
            print("Shape of the dataset before transformation : ", df.shape)
        X=df.drop(target,axis = 1)
        x_columns = list(X)
        X = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X)
        
        X = pd.DataFrame(X, columns=x_columns)
        for c in list(X):
            if len(X[c].unique()) in  [1,2]:
                if log:
                    print("Skipping Transformation for ", c, "because unique values are :",X[c].unique())
            else:
                if log:
                    print("Boxcoxing : ", c)
                X[c] = stats.boxcox(X[c])[0]
        
        
        #X = preprocessing.power_transform( X, method='box-cox')
        X = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit_transform(X)
        #X = X.values
        if log:
            print("Shape of the dataset after transformation : ", X.shape)
        return X
        
    

X = newBoxCoxTranformation(df_prep,'SalePrice',True,False)  
X,y = newBoxCoxTranformation(df_prep,'SalePrice',False,False)

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5, random_state=random.randint(1,500))#, stratify=df.BldgType)

In [73]:
reg = XGBRegressor(max_depth=5,n_estimator=100, 
                   #colsample_bytree=.25,
                   #colsample_bylevel=1,
                   reg_alpha =.5,
                   reg_lambda=0.5, 
                   learning_rate=0.1,                   
                  )
reg.fit(X_train,y_train)
reg.score(X_test,y_test)

0.875133298532814

In [74]:
np.sqrt(mean_squared_log_error(y_test, reg.predict(X_test)))

0.01097417477892352

In [75]:
np.sqrt(mean_squared_log_error(np.exp(y_test), np.exp(reg.predict(X_test))))

0.14181662582705512

In [76]:
reg.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimator=100,
       n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0.5, reg_lambda=0.5, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [77]:
reg

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimator=100,
       n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0.5, reg_lambda=0.5, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

##### We need to have different pre-processing logic to test data. We will come back to it little later.

In [78]:
def checkTheTestFile(reg):
    df_test = pd.read_csv(test_File)
    df_test['SalePrice'] = 0.0
    
    
    
    df_train =  pd.read_csv(train_File)
    df_concat = pd.concat([df_train,df_test])

    #print(df_test[df_test.TotalBsmtSF.isna()])
    #return
    df = giveMeWrangledData(df_concat,True)
    
    #print(df.info())
    df = preProcessData(df)
    df_test_dummy = df.copy()
    df_test_dummy = df_test_dummy[df_test_dummy.SalePrice == 0]
    #print(df.info())
    X = newBoxCoxTranformation(df,'SalePrice',True)
    #print(np.sqrt(mean_squared_log_error(y, reg.predict(X))))
    
    df_test_dummy['SalePrice'] = np.exp(reg.predict(X))
    #display.display(df_test_dummy[df_test_dummy['SalePrice'].isna()])
    df_test['SalePrice'] = transformTarget(df_test_dummy,True)
    #display.display(df_test[df_test['SalePrice'].isna()])
    
    return df_test, X, reg.predict(X)
df_test, X_dummy, y_dummy= checkTheTestFile(reg)

In [79]:
df_test.SalePrice.describe()

count    1459.000000
mean       11.993422
std         0.390503
min        10.668795
25%        11.739459
50%        11.948544
75%        12.238442
max        13.113459
Name: SalePrice, dtype: float64

In [80]:
df_test[df_test.SalePrice.isna()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


In [81]:
df_test[df_test.SalePrice == 0]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


In [82]:
df_test[['Id','SalePrice']]

,Id,SalePrice
0,1461,11.718786
1,1462,11.926608
2,1463,12.164692
3,1464,12.217786
4,1465,12.082026
5,1466,12.098408
6,1467,12.033278
7,1468,12.015747
8,1469,12.180098
9,1470,11.720106


In [83]:
df_test[['Id','SalePrice']].to_csv('submission.csv',index=False)

##### I got Kaggle Rank of 2539/4463 with RMSLE =0.14357
##### As on 1/17/2019 : 9:06PM IST
* 0.13501 ==> 2040 
* 0.13252 ==> 1865
* 0.13002 ==> 1704
* 0.12658 ==> 1500
* 0.12351 ==> 1250
* 0.12081 ==> 1000
* 0.11572 ==> 500
* 0.11475 ==> 250
* 0.11310 ==> 100
* 0.10985 ==> 50
* 0.10973 ==> 25
* 0.10845 ==> 10
* 0.08021 ==> 5
* 0.00000 ==> 1

##### Now that I know around what score gets what rank; can we have a function which would what would be testing score ?

##### Logic is to predict first the testing samples. Later use that for training and predict the initial training data set. We would then have actual and predicted SalePrices with which we can calculated the RMSLE.

##### Would this logic work ? let us try for our case now and compare that with Kaggle result....Finger crossed :)

In [84]:
def forCrossValidationStratifiedShuffleSplit(df):
    sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
    #print("Number of Splits configured :", sss.get_n_splits(df, df.BldgType))
    
    for train_index, test_index in sss.split(df, df.BldgType):
        yield train_index, test_index
        
    for train_index, test_index in sss.split(df, df.OverallQual):
        yield train_index, test_index

In [85]:
def crossValidationScoring(reg,X,y):
    
    return np.sqrt(mean_squared_log_error(np.exp(y), 
                                          np.exp(reg.predict(X))
                                          ))
mean_temp_rmsle = np.mean(cross_val_score(reg,X,y,cv= 5,scoring='neg_mean_squared_log_error'))
print("RMSE with without target variable transformation : ", np.sqrt(mean_temp_rmsle * -1))

mean_temp_rmsle = np.mean(cross_val_score(reg, X, y,
                                          cv= forCrossValidationStratifiedShuffleSplit(df_train),
                                          scoring=crossValidationScoring))
print("RMSE with post target variable transformation : ", mean_temp_rmsle)

RMSE with without target variable transformation :  0.010224298656041452
RMSE with post target variable transformation :  0.1435299284956907


In [86]:
def doGridSearch():
    
    start_time = datetime.datetime.now()
    
    df_train = pd.read_csv(train_File)
    df_test = pd.read_csv(test_File)
    df_test['SalePrice'] = 0
    df_concat = pd.concat([df_train,df_test])
    
    df = giveMeWrangledData(df_concat)
    df_prep = preProcessData(df)
    
    X,y = newBoxCoxTranformation(df_prep,'SalePrice',False,False)
    
    score_list = []
    for i in range(max_depth_min,max_depth_max):
        for j in range(n_estimator_min,n_estimator_max,100):
            loop_start = datetime.datetime.now()
            for bytree in [0.25, 0.5, 0.75,1]:
                for bylevel in [0.25, 0.5, 0.75,1]:

                    #reg = XGBRegressor(max_depth=i, n_estimators=j)
                    reg=XGBRegressor(max_depth=i, 
                         n_estimator=j,
                         colsample_bylevel=bylevel,
                         colsample_bytree=bytree,
                         learning_rate=.1,
                         reg_alpha =.5,
                         reg_lambda=.5,
                         n_jobs=n_jobs
                        )

                    #cv = ShuffleSplit(n_splits=20, test_size=random.randint(7,9)/10, random_state=random.randint(1,1000))
                    #cv = ShuffleSplit(n_splits=20, test_size=.5, random_state=1986)
                    #print(datetime.datetime())
                    cross_cv = cross_val_score(reg, X, y,\
                                               cv=forCrossValidationStratifiedShuffleSplit(df_train), \
                                               #cv=cv, \
                                               scoring=crossValidationScoring,n_jobs=n_jobs)
                    #print(" Validat Median Score : ", np.sqrt(np.median(cross_cv) * -1), \
                    #      "Average Score : ", np.sqrt(np.average(cross_cv) * -1) )

                    reg.fit(X,y)
                    training_score = np.sqrt(mean_squared_log_error(np.exp(y), np.exp(reg.predict(X))))
                    #print("Training Score :", training_score)

                    df_test_new, X_test, y_test = checkTheTestFile(reg)
                    reg.fit(X_test,y_test)
                    testing_score = np.sqrt(mean_squared_log_error(np.exp(y), np.exp(reg.predict(X))))
                    #print("Testing Score :", testing_score)
                    print("Scores",(training_score, np.average(cross_cv), testing_score,i,j,bytree, bylevel))
                    score_list.append((training_score, np.average(cross_cv), testing_score,i,j,bytree, bylevel))

        #print("Time for max_depth -",i,"n_estimator -",j," is : ", datetime.datetime.now() - loop_start)
    
    print("Total time for GridSearch : ", datetime.datetime.now() - start_time)
    return score_list

score_list = doGridSearch()
#sorted(score_list,key= lambda x:x[0])

Scores (0.11174865797605738, 0.14837018620403747, 0.15251715022728574, 3, 100, 0.25, 0.25)
Scores (0.10924605449357788, 0.1439643202042484, 0.15348594909989335, 3, 100, 0.25, 0.5)
Scores (0.1056298491408862, 0.14357161275128977, 0.1524730176535538, 3, 100, 0.25, 0.75)
Scores (0.10336799238692085, 0.1422378364070876, 0.1442814855843591, 3, 100, 0.25, 1)
Scores (0.1066053773146748, 0.14335644316730453, 0.14900361779272597, 3, 100, 0.5, 0.25)
Scores (0.10114557277570277, 0.14280791450533872, 0.14179841370857493, 3, 100, 0.5, 0.5)
Scores (0.09962399163235591, 0.14250586592645254, 0.14035860042884446, 3, 100, 0.5, 0.75)
Scores (0.09885432946329578, 0.14286708112863394, 0.13970358077007713, 3, 100, 0.5, 1)
Scores (0.10385049164090643, 0.1432507690467884, 0.14285430674379826, 3, 100, 0.75, 0.25)
Scores (0.09892623704308989, 0.14303820017015476, 0.14041287139127615, 3, 100, 0.75, 0.5)
Scores (0.09744890979307028, 0.14297720251340276, 0.13801650098848853, 3, 100, 0.75, 0.75)
Scores (0.097690933

Scores (0.05241178972332073, 0.14257967099115793, 0.13082707093056023, 8, 100, 1, 0.5)
Scores (0.050828291101782754, 0.1443585834115176, 0.13096718982205047, 8, 100, 1, 0.75)
Scores (0.049227667901110814, 0.14569211650974304, 0.1301535693220231, 8, 100, 1, 1)
Scores (0.07103010966828295, 0.14223036465339173, 0.1409817548170712, 9, 100, 0.25, 0.25)
Scores (0.06243014306112167, 0.14191173608579194, 0.13992714000718787, 9, 100, 0.25, 0.5)
Scores (0.06031199884294023, 0.1415598212951616, 0.14053102187062175, 9, 100, 0.25, 0.75)
Scores (0.05948369483249965, 0.14038411569796053, 0.1356545041561413, 9, 100, 0.25, 1)
Scores (0.06162273514276969, 0.14075818687804492, 0.13609969473277977, 9, 100, 0.5, 0.25)
Scores (0.05334024613418082, 0.14249393333019483, 0.13419767637202068, 9, 100, 0.5, 0.5)
Scores (0.05261998982937355, 0.14199701018229877, 0.13348286928362943, 9, 100, 0.5, 0.75)
Scores (0.05136376854658579, 0.1421093547950955, 0.13129109184227872, 9, 100, 0.5, 1)
Scores (0.05626416320944306,

def doGridSearch():
    
    start_time = datetime.datetime.now()
    
    df_train = pd.read_csv(train_File)
    df_test = pd.read_csv(test_File)
    df_test['SalePrice'] = 0
    df_concat = pd.concat([df_train,df_test])
    
    df = giveMeWrangledData(df_concat)
    df_prep = preProcessData(df)
    
    X,y = newBoxCoxTranformation(df_prep,'SalePrice',False,False)
    
    score_list = []
    '''for i in range(max_depth_min,max_depth_max):
        for j in range(n_estimator_min,n_estimator_max,100):
            loop_start = datetime.datetime.now()
            for bytree in [0.25, 0.5, 0.75,1]:
                for bylevel in [0.25, 0.5, 0.75,1]:'''
    for reg_alpha in [.1, .2, .3, .4, .5, .6, .7, .8, .9, 1]:
        #loop_start = datetime.datetime.now()
        for reg_lambda in [.1, .2, .3, .4, .5, .6, .7, .8, .9, 1]:

        #for lrate in [.01, .025, 0.05, .075, .1, .25, .5]:
            #j=100


            #reg = XGBRegressor(max_depth=i, n_estimators=j)
            '''reg=XGBRegressor(max_depth=i, 
                 n_estimator=j,
                 colsample_bylevel=bylevel,
                 colsample_bytree=bytree,
                 learning_rate=.1,
                 reg_alpha =1,
                 reg_lambda=1 ,
                 n_jobs=n_jobs
                )'''

            i=6
            j=100
            bylevel= 1
            bytree = .25
            reg=XGBRegressor(max_depth=6, 
                 n_estimator=100,
                 colsample_bylevel=bylevel,
                 colsample_bytree=bytree,
                 learning_rate=.1,
                 reg_alpha =reg_alpha,
                 reg_lambda=reg_lambda ,
                 n_jobs=n_jobs
                )

            #cv = ShuffleSplit(n_splits=20, test_size=random.randint(7,9)/10, random_state=random.randint(1,1000))
            #cv = ShuffleSplit(n_splits=20, test_size=.5, random_state=1986)
            #print(datetime.datetime())
            cross_cv = cross_val_score(reg, X, y,\
                                       cv=forCrossValidationStratifiedShuffleSplit(df_train), \
                                       #cv=cv, \
                                       scoring=crossValidationScoring,n_jobs=n_jobs)
            #print(" Validat Median Score : ", np.sqrt(np.median(cross_cv) * -1), \
            #      "Average Score : ", np.sqrt(np.average(cross_cv) * -1) )

            reg.fit(X,y)
            training_score = np.sqrt(mean_squared_log_error(np.exp(y), np.exp(reg.predict(X))))
            #print("Training Score :", training_score)

            df_test_new, X_test, y_test = checkTheTestFile(reg)
            reg.fit(X_test,y_test)
            testing_score = np.sqrt(mean_squared_log_error(np.exp(y), np.exp(reg.predict(X))))
            #print("Testing Score :", testing_score)
            print("Scores",(training_score, np.average(cross_cv), testing_score,i,j,bytree, bylevel, reg_alpha, reg_lambda))
            score_list.append((training_score, np.average(cross_cv), testing_score,i,j,bytree, bylevel,reg_alpha, reg_lambda))

        #print("Time for max_depth -",i,"n_estimator -",j," is : ", datetime.datetime.now() - loop_start)
    
    print("Total time for GridSearch : ", datetime.datetime.now() - start_time)
    return score_list

score_list = doGridSearch()
#sorted(score_list,key= lambda x:x[0])

In [ ]:
temp_df = pd.DataFrame(score_list,columns=["training_score",
                                           "validation_score",
                                           "testing_score", 
                                           "max_depth",
                                           "n_estimator",
                                           "bytree",
                                           "bylevel",
                                           "reg_alpha",
                                           "reg_lambda"
                                          ])

temp_df.to_csv("GridSearchResults-"+str(datetime.datetime.now().date()))
temp_df

In [ ]:
temp_df[temp_df.testing_score == temp_df.testing_score.min()]

In [ ]:
temp_df[temp_df.validation_score == temp_df.validation_score.min()]

In [ ]:
def plottheTemp(df,title):
    #for bytree in [.25,.5,.75,1]:
    #for n in temp_df.n_estimator.unique():
        fig, ax = plt.subplots()
        plt.title(title)
        ax = df.plot.line('max_depth','training_score',ax =ax,xticks= list(range(3,11)))
        ax = df.plot.line('max_depth','validation_score',ax =ax,xticks= list(range(3,11)))
        ax = df.plot.line('max_depth','testing_score',ax =ax,xticks= list(range(3,11)))
        #x_point = temp_df[temp_df.validation_score == temp_df.validation_score.min()]['max_depth'].tolist()[0]
        #ax= ax.vlines(x_point,0,.2)
        #x_point1 = df[df.testing_score == temp_df.testing_score.min()]['max_depth'].tolist()[0]
        #ax= ax.vlines(x_point1,0,.2)
        
        plt.show()
#,'validation_score')

for bytree in temp_df.bytree.unique():
    plottheTemp(temp_df[temp_df.bytree == bytree], "bytree - "+str(bytree))
    
for bylevel in temp_df.bylevel.unique():
    plottheTemp(temp_df[temp_df.bylevel == bylevel], "bylevel - "+str(bylevel))

In [ ]:
temp_df[["reg_alpha","validation_score"]].groupby(by="reg_alpha").agg('mean').reset_index()

In [ ]:
temp_df[["reg_lambda","validation_score"]].groupby(by="reg_lambda").agg('median').reset_index()

##### Kaggle score remained unchanged, this proves that our logic of test score calculation fails and there must be some value prediction which is huge which brings the average error value up


##### Next Steps:
* re run the grid search and note training score, validation score and testing score. This should not only double check on test score calculation but also gives us the right hyper parameter from the training and validation perspective.
* address the runtime error during scaling or power transform. If boxcox fails attempt a log transformation at least.
* stratify fold testing to check if the training score & validation in the previous exercise still holds goods.
* hyper parameter research for XGBoost
* target variable transformation
* best of best stack approach
* team work stack approach
* XGBoost as the final assesser in best of best stack approach
* XGBoost as the final assesser in the team work starck approach
* 3 layers in stack approach: best of best candidates in the order of their accuracy feeding on input in each case.
* re-assess the dataset

##### The grid search in the kaggle resulted in the different hyper parameter for lowest validation score. Not sure why is that ? validation is through the shuffle split. isnt 3 cross validation set sufficient ?

##### Or it is a game of kfold shuffle split and stratify ?

##### It only makes sense to move on for stacked approach and other hyper parameter tuning if we sort out the cross validation consistency issue. Otherwise, we cannot have the confidence of impact of changes in stacked approaches.

##### The full grid search for split count = 10 is as below. It is evident that 
* testing training score is of no use. We can ignore it in the future grid searches. it would save couple minutes from one grid search.

##### Following are the observation yet to be confirmed:
* Does the testing score confidence is high; post custom stratified split ? We will have predict with the best testing score and see the rank in Kaggle. 6,800 = > 0.15404  & 3,800 ==> 0.16021
* Does we have relation between validation score and testing score ?
* Can we first focus on training score to be 0 with other hyper parameters ? would that be overfitting ? Should we need still learning curve of validation vs testing score.?
* Can we ignore n_estimator and see if the validation score converges with testing score only with max_depth and other hyper parameters?

##### Let us talk about the hyper parameter till we get the gridsearch result

* https://xgboost.readthedocs.io/en/latest/parameter.html
* https://www.kaggle.com/dansbecker/xgboost
* https://www.datacamp.com/community/tutorials/xgboost-in-python

* 1 being the max value, let us have half as the value for {'colsample_bytree':0.5, 'colsample_bylevel':0.5}
* learning_rate = 0.05 because we have already using 0.1 so far. It is suggested in the kaggle blog.
* n_jobs= 2 /4 based on the cpu. I guess Kaggle provides 4 cpu machine. I saw the max cpu spike as 400%
* Surprisingly and unnoticed so far that it does the cross validation by itself. so we can leverage the n_estimator to be 1000 and use early_stopping_rounds for our quick turnaround. So our 2 for loops reduced to one :)


In [ ]:
reg

In [ ]:
reg = XGBRegressor(max_depth=4, n_estimators=1000, learning_rate = 0.05, n_jobs = 2, colsample_bylevel = 0.5, colsample_bytree = 0.5)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5, random_state=random.randint(1,500))
reg.fit(X_train,y_train, early_stopping_rounds=250, eval_metric='rmse',eval_set=[(X_test,y_test)], verbose=100,)

'''my_model = XGBRegressor(n_estimators=1000)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=False)'''
print()

* 15 ==> validation_0-rmse:0.145474
* 25 ==> [ 527 ]	validation_0-rmse:0.14391
* 100 ==> [567]	validation_0-rmse:0.143869
* 250 ==> [567]	validation_0-rmse:0.143869


* for max_depth = 4
  * [273]	validation_0-rmse:0.148329
  * [729]	validation_0-rmse:0.12575


In [ ]:
reg2 = XGBRegressor()
reg2

##### n_estimator we will choose it at the end again through Stratified

##### With hyper parameter set to above the grid search is as below: